In [ ]:
#!pip install particle

In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd

from particle import Particle

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

##josie laptop 
#topdir = "/Users/josieswann/Desktop/important documents"

# On Bellis' laptop
#topdir = '/home/bellis/babar_data/bnv_plambda/'

# At Bellis' home
topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")
IS_MC=True


#'''
# Collision data
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'

start = time.time()

data_collision = ak.from_parquet(filename)
#data_collision = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

print(type(data_collision))
#'''


In [ ]:
### information about cross section --> what we'll use to calculate scaling values for histograms 

dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

In [ ]:
sp = data['spmode']

np.unique(sp.to_list())

In [ ]:
region_definitions

In [ ]:
dcuts = bat.get_final_masks(data, region_definitions=region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

In [ ]:
# Make our histograms
all_hists = bat.create_empty_histograms(hist_defs)

bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000
weights['0'] = 1

print(weights)
print()
print(spmodes)

In [ ]:
# Pull out some SP mode
spmode = '1005'
#spmode='998'
#spmode='991'
#spmode='-999'
#spmode='0'
#mask_sp = data['spmode'] == spmode

mask_sp = (data['spmode'] == '998') | (data['spmode'] == '1005')

mask_event = dcuts[-1]['event']


# Collision data
#mask_sp = data['spmode']=='0'

# Make a subset
data_sp = data[mask_sp & mask_event]

# For data we need to account for blinding
'''
if IS_MC is False:
    nB = data_sp['nB']
    Bp3 = data_sp['Bp3']
    nBp3 = ak.num(Bp3)
    
    mask_to_account_for_blinding = nB == nBp3
    print(len(nB[mask_to_account_for_blinding]), len(nB[~mask_to_account_for_blinding]))
    
    data_sp = data_sp[mask_to_account_for_blinding]
'''


# Make the plot
mes =    ak.flatten(data_sp['BpostFitMes'])
DeltaE = ak.flatten(data_sp['BpostFitDeltaE'])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data_sp['Lambda0_unc_Mass'])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

In [ ]:
def dump_truth_information(data, n=0, verbose=0):
    
    if verbose:
        print("The MC and tracks for the first entry")
        idx = data['TRKMCIdx'][n]
        mclund = data['mcLund'][n]

        for i,id in enumerate(idx):
            print(f"{i:2d}  {id:4d}   {mclund[id]:>6}")
    print()

    return 1

##########################################################################
# Pull out some SP mode
#spmode = '1005'
#spmode='998'
#spmode='991'
spmode='-999'
#spmode='0'
mask_sp = data['spmode'] == spmode
mask_event = dcuts[-1]['event']


dump_truth_information(data[mask_sp & mask_event],n=10,verbose=1)

In [ ]:
data.fields

In [ ]:
#n = 0
#n = 41059
#n = 40713
n = 11802
#n = 2919
print(data[n]['mcLen'])
print(data[n]['mothIdx'])
print(data[n]['mcLund'])
print(data[n]['dauLen'])
print(data[n]['dauIdx'])

idxs_printed = []

idx = data[n]['TRKMCIdx']
mclund = data[n]['mcLund']
nTRK = data[n]['nTRK']

print("\nThe MC and tracks for the first entry")
        
print(f"nTRK: {nTRK}")
for i,id in enumerate(idx):
    name = "NONE"
    try:
        name = Particle.from_pdgid(mclund[id]).name
    except:
        name = "NONE"

    print(f"{i:2d}  {id:4d}   {mclund[id]:6d}    {name}")
print()


print(f"SPMODE: {data[n]['spmode']} ----------------------------")

for i in range(data[n]['mcLen']):

    mothIdx,mcLund,dauLen,dauIdx = data[n]['mothIdx'][i], data[n]['mcLund'][i], data[n]['dauLen'][i], data[n]['dauIdx'][i]
    
    name = "NONE"
    try:
        name = Particle.from_pdgid(mcLund).name
    except:
        name = "NONE"

    if i not in idxs_printed:
        print(f"{i:3d}  mothIdx:{mothIdx:3d}  mcLund:{mcLund:6d} {name:12s}   ndau: {dauLen:2d}  dauIdx: {dauIdx}")
        if dauLen>0:
            for j in range(dauLen):

                mothIdx2,mcLund2,dauLen2,dauIdx2 = data[n]['mothIdx'][j+dauIdx], data[n]['mcLund'][j+dauIdx], data[n]['dauLen'][j+dauIdx], data[n]['dauIdx'][j+dauIdx]

                try:
                    name2 = Particle.from_pdgid(mcLund2).name
                except:
                    name2 = "NONE"

                if j+dauIdx not in idxs_printed:
                    print(f"\t{j+dauIdx:3d}  mothIdx:{mothIdx2:3d}  mcLund:{mcLund2:6d} {name2:12s}   ndau: {dauLen2:2d}  dauIdx: {dauIdx2}")
                    idxs_printed.append(j+dauIdx)
            
        idxs_printed.append(i)

In [ ]:
#Particle.from_pdgid(10022)
def print_gen_particle(data, i=0, depth=0, ngen=0, idxs_printed=None, in_loop=False, debug=False):

    #print(i,ngen,idxs_printed)

    if len(idxs_printed)==ngen:
        return ngen

    if i not in idxs_printed:
        mothIdx,mcLund,dauLen,dauIdx = data['mothIdx'][i], data['mcLund'][i], data['dauLen'][i], data['dauIdx'][i]
        
        tabs = depth*"   "
        
        name = "NONE"
        try:
            name = Particle.from_pdgid(mcLund).name
        except:
            name = "NONE"

        if debug:
            print(f"{tabs}{i:<3d}  mothIdx:{mothIdx:3d}  mcLund:{mcLund:6d} {name:12s}   ndau: {dauLen:2d}  dauIdx: {dauIdx}")
        else:
            print(f"{tabs} {name:12s}")

        idxs_printed.append(i)

        if dauLen>0:
            for j in range(dauLen):
                #print(dauLen,j+dauIdx)
                print_gen_particle(data, i=j+dauIdx, idxs_printed=idxs_printed, ngen=ngen, depth=depth+1, in_loop=True, debug=debug)
            #print("Done with loop")
            return 1
        else:
            #return 1
            if i+1>=ngen:
                return ngen
            elif in_loop is False:
                print_gen_particle(data, i=i+1, idxs_printed=idxs_printed, ngen=ngen, depth=depth, debug=debug)        

    '''
    if dauIdx<0 and i>5:
        return i+1
    else:
        print("recursion")
        print_gen_particle(data, i=i+1, idxs_printed=idxs_printed, ngen=ngen, depth=depth)        

    return i+1
    '''
    
############################
n = 40713
#n = 41696
#n = 41965
#n = 8519
#n = 11802
print(f"SPMODE: {data[n]['spmode']} ----------------------------")

idxs_printed = []
ngen = data[n]['mcLen']
print_gen_particle(data[n], i=0, idxs_printed=idxs_printed, ngen=ngen, depth=0)

In [ ]:
spmode='1005'
#spmode='0'
mask_sp = data['spmode'] == spmode
mask_event = dcuts[-1]['event']

n = 0

dump_truth_information(data[mask_sp & mask_event],n=n,verbose=1)

idxs_printed = []
ngen = data[n]['mcLen']
print_gen_particle(data[mask_sp & mask_event][n], i=0, idxs_printed=idxs_printed, ngen=ngen, depth=0)

In [ ]:
lo = 0
hi = 2

a = data[lo:hi]['runNumber']
b = data[lo:hi]['upperID']
c = data[lo:hi]['lowerID']
d = data[lo:hi]['majorID']
print(a)
print(b)
print(c)
print(d)
#data.fields

In [ ]:
lid = data['lowerID']
lid_unique = np.unique(lid.to_list())

print(len(lid), len(lid_unique))

In [ ]:
lid = data['lowerID']

#mask = data['lowerID'] == 1613932672
#mask = data['lowerID'] == -1483012466
#mask = data['lowerID'] == -971300253
#mask = data['lowerID'] == 332077398
#mask = data['lowerID'] == 705777929
#mask = data['lowerID'] == 958485991
mask = data['lowerID'] == 1023553332

print(len(data))
print(len(data[mask]))


'''
idxs_printed = []
ngen = data[mask]['mcLen'][0]
print(ngen)
print_gen_particle(data[mask][0], i=0, idxs_printed=idxs_printed, ngen=ngen, depth=0)
''';

num = ak.local_index(data, axis=0)
print(lid)
print(num)
print(num[mask])
#data[mask]

n = 41965
print(data[n]['lowerID'])
print(data[n]['spmode'])

In [ ]:
#data[0]
print("hi",6*"\t","bye")

In [ ]:
? Particle.from_pdgid

In [ ]:
a = Particle.from_pdgid(2122)
print(type(a))

print(a.name)

a

# Recursion

In [ ]:
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5))  


In [ ]:
def fibonacci(n):
    # Base cases
    if n == 0:
        return 0
    elif n == 1:
        return 1
    # Recursive case
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Example usage
print(fibonacci(10))

In [ ]:
def tail_fact(n, acc=1):
    # Base case
    if n == 0:
        return acc
    # Tail recursive call with an accumulator
    else:
        return tail_fact(n-1, acc * n)

def nontail_fact(n):
    # Base case
    if n == 1:
        return 1
    # Non-tail recursive call because the multiplication happens after the call
    else:
        return n * nontail_fact(n-1)

# Example usage
print(tail_fact(5))  
print(nontail_fact(5))

In [ ]:
>>> def countdown(n):
...     print(n)
...     if n == 0:
...         return             # Terminate recursion
...     else:
...         countdown(n - 1)   # Recursive call
...

>>> countdown(5)
5
4
3
2
1
0